In [ ]:

%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
from braindecode.datasets.bbci import BBCIDataset

# we loaded all sensors to always get same cleaning results independent of sensor selection
# There is an inbuilt heuristic that tries to use only EEG channels and that definitely
# works for datasets in our paper
#train_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/13.mat')
#test_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/test/13.mat')
cnt = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/4.mat',
                 load_sensor_names=['C3', 'C4']).load()
cnt = cnt.drop_channels(['STI 014'])

In [ ]:
from collections import OrderedDict
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne

marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2],),
                         ('Rest', [3]), ('Feet', [4])])
ival = [500,1500]
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import exponential_running_standardize

log.info("Resampling train...")
cnt = resample_cnt(cnt, 250.0)
log.info("Standardizing train...")
cnt = mne_apply(lambda a: exponential_running_standardize(a.T ,factor_new=1e-3, init_block_size=1000, eps=1e-4).T,
                     cnt)
cnt = resample_cnt(cnt, 2.0)
train_set = create_signal_target_from_raw_mne(cnt, marker_def, ival)

In [ ]:
x_right = train_set.X[train_set.y == 0]

x_rest = train_set.X[train_set.y == 2]

In [ ]:
inputs = np_to_var(x_rest[:10,0,0,], dtype=np.float32)

In [ ]:
fake_inputs = (th.rand(10) * 0.2).detach().requires_grad_(True)



In [ ]:
plt.plot(var_to_np(inputs), var_to_np(inputs) * 0, ls='', marker='o')
plt.plot(var_to_np(fake_inputs), var_to_np(fake_inputs) * 0, ls='', marker='o')

### energy distance

In [ ]:
diffs = []
for _ in range(10000):
    xa = inputs[th.randperm(len(inputs))[:3]]
    xb = inputs[th.randperm(len(inputs))[:3]]
    ya = fake_inputs[th.randperm(len(fake_inputs))[:3]]
    yb = fake_inputs[th.randperm(len(fake_inputs))[:3]]

    xab = th.mean(th.abs(xa.unsqueeze(1) - xb.unsqueeze(0)))
    xy = th.mean(th.abs(xa.unsqueeze(1) - ya.unsqueeze(0)))
    yab = th.mean(th.abs(ya.unsqueeze(1) - yb.unsqueeze(0)))

    diffs.append((2 * xy - xab -yab).item())

In [ ]:
np.sqrt(np.mean(diffs))

In [ ]:
import scipy.stats
scipy.stats.energy_distance(var_to_np(inputs), var_to_np(fake_inputs))

In [ ]:
scipy.stats.energy_distance([0,0], [1,3])

In [ ]:
scipy.stats.energy_distance([0,0], [2,2])

In [ ]:
np.sqrt(np.sum((np.array([0,0]) - np.array([1,2])) ** 2))

In [ ]:
u_values = np.array([0,3,7,13,20])
v_values = np.array([1,2,4,6,7])

In [ ]:
u_sorter = np.argsort(u_values)
v_sorter = np.argsort(v_values)

all_values = np.concatenate((u_values, v_values))
all_values.sort(kind='mergesort')

# Compute the differences between pairs of successive values of u and v.
deltas = np.diff(all_values)

In [ ]:
u_cdf_indices = u_values[u_sorter].searchsorted(all_values[:-1], 'right')
v_cdf_indices = v_values[v_sorter].searchsorted(all_values[:-1], 'right')

In [ ]:
u_cdf

In [ ]:
v_cdf

In [ ]:
u_cdf = u_cdf_indices / u_values.size

v_cdf = v_cdf_indices / v_values.size

In [ ]:
np.sqrt(np.sum(np.multiply(np.square(u_cdf - v_cdf), deltas))) * np.sqrt(2)

In [ ]:
scipy.stats.energy_distance(u_values, v_values)